In [21]:
import re
from urllib.request import urlopen, urlretrieve, Request
from urllib.parse import urljoin
from urllib.error import HTTPError, URLError

from bs4 import BeautifulSoup
import pandas as pd


def url2soup(url):
    """Fetch a webpage a return a BeautifulSoup soup object from its HTML"""
    try:
        request = Request(url, headers={'User-agent' : 'Magic Browser'})
        html_handler = urlopen(request)
        html = html_handler.read()
        soup = BeautifulSoup(html, 'html.parser')
    except (HTTPError, URLError, ConnectionResetError) as e:
        print('Error fetching {} : {}'.format(url.lower(), e))
        soup = None
    return soup

In [22]:
def parse_hotslogs_table(soup):
    # Find the tables with a body
    tables_soup = [soup for soup in soup.find_all('table') if soup.find('tbody')]
    assert len(tables_soup) == 1
    table_soup = tables_soup[0]
    header_soup = table_soup.find('thead').find('tr')
    header = [field.text for field in table_soup.find_all('th')]
    body_soup = soup.find('tbody')
    body = []
    for row in body_soup.find_all('tr'):
        fields = [field.text for field in row.find_all('td')]
        body.append(fields)
    return header, body

# Winrates

In [23]:
url = 'https://www.hotslogs.com/Sitewide/HeroAndMapStatistics'
soup = url2soup(url)
header, body = parse_hotslogs_table(soup)
df_winrate = pd.DataFrame(data=body, columns=header).set_index('Hero')

# Tier list

In [24]:
url = 'https://www.tentonhammer.com/articles/heroes-of-the-storm-tier-list-quickmatch'
soup = url2soup(url)

In [25]:
tables = soup.find_all('table')

In [26]:
tenton2hotslogs = {
    'T.L.V': 'The Lost Vikings',
    'E.T.C': 'E.T.C.',
    'Lucio': 'Lúcio',
    "Cho'gall": 'Cho',
}

# Add new columns
df_winrate['Tier num'] = None
df_winrate['Tier'] = None
df_winrate['Role'] = None
df_winrate['Difficulty'] = None
for i, table in enumerate(tables):
    header = table.find('thead')
    tier = header.find_all('th')[0].text.replace('\n', '')
    rows = table.find('tbody').find_all('tr')
    for row in rows:
        fields = row.find_all('td')
        name = re.sub("[^a-zA-Z-. ']*", '', fields[0].text)
        name = tenton2hotslogs.get(name, name)
        role = fields[2].text
        difficulty = fields[3].text
        
        df_winrate.loc[name, 'Tier num'] = i + 1
        df_winrate.loc[name, 'Tier'] = tier
        df_winrate.loc[name, 'Role'] = role
        df_winrate.loc[name, 'Difficulty'] = difficulty

In [27]:
df_winrate = df_winrate.sort_values(by=['Tier num', 'Win Percent'], ascending=[True, False])
df_winrate[
    (df_winrate['Tier num'] <= 4)
    #& ((df_winrate['difficulty'] == 'Easy') | (df_winrate['difficulty'] == 'Medium'))
    #& ((df_winrate['role'] == 'Specialist'))
]

,,Games Played,Games Banned,Popularity,Win Percent,% Δ,,,,Tier num,Tier,Role,Difficulty
Hero,,,,,,,,,,,,,
Sonya,,8995,1106,22.8 %,53.0 %,1.6 %,Warrior,Bruiser,"Sonia,Barbare,Barbarian,Barbarin,Bárbara,Соня,...",1,S+ Tier,Warrior,Medium
Varian,,14803,6741,48.7 %,49.4 %,-1.3 %,Assassin,Bruiser,"瓦里安,바리안,Вариан,바리인",1,S+ Tier,Tank/Bruiser/Assassin,Medium
Valeera,,2429,2011,10.0 %,45.3 %,0.1 %,Assassin,Ambusher,"Valira,Валира,瓦莉拉,瓦麗拉,발리라",1,S+ Tier,Assassin,Hard
Samuro,,2983,1043,9.1 %,55.3 %,2.5 %,Assassin,Ambusher,"萨穆罗,薩姆羅,Самуро,사무로,Samura",2,S Tier,Assassin,Medium
Ragnaros,,3735,882,10.4 %,53.1 %,0.0 %,Assassin,Sustained Damage,"拉格納羅斯,拉格纳罗斯,라그나로스,Рагнарос",2,S Tier,Assassin,Medium
Azmodan,,7553,3118,24.1 %,52.9 %,-0.4 %,Specialist,Siege,"Asmodan,Азмодан,Azmodán,Azmadan,아즈모단,阿兹莫丹,阿茲莫丹",2,S Tier,Specialist,Easy
Rehgar,,10646,1360,27.1 %,52.1 %,0.7 %,Support,Healer,"РЕГАР,레가르,雷加尔,雷加",2,S Tier,Support,Medium
Kael'thas,,14179,6517,46.8 %,51.1 %,-0.6 %,Assassin,Burst Damage,"Kael’thas,캘타스,凱爾薩斯,凯尔萨斯,Кель'тас",2,S Tier,Assassin,Medium
Malfurion,,13662,6039,44.5 %,50.8 %,0.5 %,Support,Healer,"МАЛФУРИОН,말퓨리온,玛法里奥,瑪法里恩",2,S Tier,Support,Medium


# Duos

In [28]:
import time

from selenium import webdriver

In [29]:
def get_duo_table_soup(driver, url):
    driver.get(url)

    dropdown_button = driver.find_element_by_xpath(
        '//*[@id="ctl00_MainContent_DropDownGameMode"]/span/span[2]'
    )
    dropdown_button.click()
    quickmatch_button = driver.find_element_by_xpath(
        '//*[@id="ctl00_MainContent_DropDownGameMode_DropDown"]/div/ul/li[4]'
    )
    quickmatch_button.click()
    duos_button = driver.find_element_by_link_text('Duos')
    duos_button.click()


    # This while loop is not very useful because we actually need to wait
    # for the data to be loaded afterwards.
    game_mode = ''
    while game_mode != 'Quick Match':
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        game_mode = soup.find('div', {'id': 'ctl00_MainContent_DropDownGameMode'}).text.strip()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup.find(
        'div',
        {'id': 'winRateWithOtherHeroes'}
    )

In [30]:
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference('permissions.default.image', 2)
firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')

driver = webdriver.Firefox(firefox_profile=firefox_profile)

df_duos = None
for hero in list(df_winrate.index):
    print(hero)
    url = 'https://www.hotslogs.com/Sitewide/HeroDetails?Hero=' + hero
    table_soup = get_duo_table_soup(driver, url)
    header, body = parse_hotslogs_table(table_soup)
    header = ['Hero'] + header
    body = [[hero] + row for row in body]
    df = pd.DataFrame(data=body, columns=header)
    df_duos = pd.concat([df_duos, df])
driver.close()

Sonya
Varian
Valeera
Samuro
Ragnaros
Azmodan
Rehgar
Kael'thas
Malfurion
Anub'arak
Johanna
Gul'dan
Sgt. Hammer
Falstad
Lúcio
Muradin
Zarya
Thrall
Cho
Genji
Malthael
Nazeebo
Zagara
Arthas
Kerrigan
Nova
Li Li
Sylvanas
Brightwing
Dehaka
Illidan
Zul'jin
Lunara
Tyrande
D.Va
E.T.C.
Zeratul
Lt. Morales
Cassia
Li-Ming
Tychus
Medivh
Murky
Tracer
Uther
Artanis
Jaina
Gazlowe
Leoric
Greymane
Valla
Diablo
Kharazim
Tyrael
Stitches
Probius
Tassadar
Alarak
Raynor
Chromie
Auriel
Abathur
Butcher
Xul
Rexxar
The Lost Vikings
Chen
The Butcher
Stukov
Garrosh
Gall


In [31]:
df_duos = df_duos.sort_values(by='Win Percent With', ascending=False)
df_duos[
    df_duos['Games Played With'].str.replace(',', '').astype(int) > 100
].head(30)

,Hero,,Team Hero,Games Played With,Win Percent With,,,
0,Rexxar,,Arthas,110,68.2 %,Warrior,Bruiser,"АРТАС,아서스,阿尔萨斯,阿薩斯,Артас"
0,Gall,,Xul,315,67.6 %,Specialist,Siege,"蘇爾,祖尔,줄,Зул"
1,Cho,,Xul,357,66.7 %,Specialist,Siege,"蘇爾,祖尔,줄,Зул"
0,The Butcher,,Xul,"1,430",66.6 %,Specialist,Siege,"蘇爾,祖尔,줄,Зул"
4,The Lost Vikings,,Rehgar,122,66.4 %,Support,Healer,"РЕГАР,레가르,雷加尔,雷加"
1,The Butcher,,Ragnaros,"1,084",66.3 %,Assassin,Sustained Damage,"拉格納羅斯,拉格纳罗斯,라그나로스,Рагнарос"
2,The Butcher,,Murky,"1,666",65.7 %,Specialist,Utility,"Bourbie,Męcik,Fosky,МУРЧАЛЬ,Murquinho,머키,鱼人,奔波..."
0,Xul,,The Butcher,"1,243",65.6 %,Assassin,Ambusher,"El Carnicero,O Açougueiro,屠夫,도살자,Der Schlächte..."
3,The Butcher,,Azmodan,"2,626",65.4 %,Specialist,Siege,"Asmodan,Азмодан,Azmodán,Azmadan,아즈모단,阿兹莫丹,阿茲莫丹"
1,Xul,,Azmodan,"1,054",65.3 %,Specialist,Siege,"Asmodan,Азмодан,Azmodán,Azmadan,아즈모단,阿兹莫丹,阿茲莫丹"
